In [1]:
import os
script_name = 'Test_20180928.ipynb'
dir_path = os.path.dirname(os.path.realpath(script_name))
parent_path = os.path.dirname(dir_path)
Func_path = os.path.join(parent_path,'Func')

import sys
sys.path.append(Func_path)

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from loading_data import DataLoader_base
from building_network import NetworkBuild_base

Using TensorFlow backend.


In [3]:
dataset = DataLoader_base(data_source="inside",filename="cifar10")

In [5]:
network = NetworkBuild_base(task="classification", input_shape=[224,224,3], 
                            output_shape = 10, pre_train=False, net_name="VGG19", customized_model=None)

In [ ]:
test_model = network.model
print(test_model.summary())

In [6]:
print(network.model.summary())
print(len(network.model.layers))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 224, 224, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 222, 222, 32)      9248      
_________________________________________________________________
activation_2 (Activation)    (None, 222, 222, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 32)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 111, 111, 64)      18496     
__________

In [ ]:
from keras.models import Sequential
# network.model.layers.pop()
# print(network.model.)
# print(network.model.summary())

In [ ]:
print(network.model.output_shape[-1])

In [ ]:
tmp_model = Sequential()
tmp_model.add(Input(shape=[224,224,500]))
if network.model.output_shape[-1] == 512:
    tmp_model.add(network.model.layers[-1])
    network.model.layers.pop()

In [ ]:
from keras.models import Model
from keras.layers import Input
sub_model_1 = Input(shape=[100,100,512])
sub_model_2 = network.model.layers[-1]
new_model = Model(inputs=sub_model_1,outputs=sub_model_2.output)
print(new_model.summary())

In [ ]:
dataset = DataLoader_base(data_source="inside",filename="mnist")

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [ ]:
	def BuildModel(input_shape,output_shape):
		model = Sequential()
		model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
		model.add(Activation('relu'))
		model.add(Conv2D(32, (3, 3)))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		model.add(Conv2D(64, (3, 3), padding='same'))
		model.add(Activation('relu'))
		model.add(Conv2D(64, (3, 3)))
		model.add(Activation('relu'))
		model.add(MaxPooling2D(pool_size=(2, 2)))
		model.add(Dropout(0.25))

		model.add(Flatten())
		model.add(Dense(512))
		model.add(Activation('relu'))
		model.add(Dropout(0.5))
		model.add(Dense(output_shape))
		model.add(Activation('softmax'))
		return model

In [ ]:
del tmp
tmp = BuildModel(network.input_shape,network.output_shape)

In [ ]:
print(tmp.summary())